In [54]:
from bs4 import BeautifulSoup
import requests
import json

In [52]:
# Constants
API_URL = 'http://localhost:5000'

BASE_URL = "https://www.imdb.com"

MAX_RESULT_PER_PAGE = 250
URL = f"{BASE_URL}/search/name/?adult=include&count=250"

TOTAL = 11_922_407


In [11]:
r = requests.get(URL + 'start=1')
soup = BeautifulSoup(r.content, 'html.parser')

In [67]:
def get_full_img_url(url):
    """
    'https://m.media-amazon.com/images/M/MV5BMjA4NDkyODA3M15BMl5BanBnXkFtZTgwMzUzMjYzNzM@._V1_UY209_CR34,0,140,209_AL_.jpg'
    to
    https://m.media-amazon.com/images/M/MV5BMjA4NDkyODA3M15BMl5BanBnXkFtZTgwMzUzMjYzNzM@.jpg
    """
    base_path = url.split('._V1_')[0]
    ext = url.split('.')[-1]
    return base_path + '.' + ext
    
get_full_img_url('https://m.media-amazon.com/images/M/MV5BMjA4NDkyODA3M15BMl5BanBnXkFtZTgwMzUzMjYzNzM@._V1_UY209_CR34,0,140,209_AL_.jpg')

'https://m.media-amazon.com/images/M/MV5BMjA4NDkyODA3M15BMl5BanBnXkFtZTgwMzUzMjYzNzM@.jpg'

In [68]:
items = soup.find_all(attrs={"class": "lister-item mode-detail"})

item = items[0]
image_url = get_full_img_url(item.img.get('src'))
actor_page_url = BASE_URL + item.a.get('href')
actor_page_url

'https://www.imdb.com/name/nm1720028'

In [22]:
actor_page_r = requests.get(actor_page_url)
actor_page_soup = BeautifulSoup(actor_page_r.content, 'html.parser')

In [69]:
# Name
name = actor_page_soup.title.text.replace(' - IMDb', '')
firstname, *middlename, lastname = name.split(' ')
middlename = ' '.join(middlename)

# Birth date and place
born_info = actor_page_soup.find(id='name-born-info')
birth = born_info.time.get('datetime')
birth_place = born_info.find_all('a')[-1].text

data = {
    "name": name,
    "firstname": firstname,
    "middlename": middlename,
    "lastname": lastname,
    "birth": birth,
    "birth_place": birth_place,
    "source": actor_page_url,
    "image": image_url,
}

data

{'name': 'Amber Heard',
 'firstname': 'Amber',
 'middlename': '',
 'lastname': 'Heard',
 'birth': '1986-4-22',
 'birth_place': 'Austin, Texas, USA',
 'source': 'https://www.imdb.com/name/nm1720028',
 'image': 'https://m.media-amazon.com/images/M/MV5BMjA4NDkyODA3M15BMl5BanBnXkFtZTgwMzUzMjYzNzM@.jpg'}

In [83]:
def get_or_create_profile(data):
    r = requests.get(
        API_URL + f'/profiles?sort=created_date,desc&filter.name=$eq:{data["name"]}'
    )
    res = json.loads(r.text)
    if int(res["meta"]["total"]) >= 1:
        return res["data"][0]
    
    r = requests.post(
        API_URL + "/profiles",
        json={
            "name": data["name"],
            "birth": data["birth"],
            "first_name": data["firstname"],
            "last_name": data["lastname"],
            "middle_name": data["middlename"],
            "attributes": {
                "birth_place": data["birth_place"],
                "imdb": data["source"],
            },
        },
    )
    return json.loads(r.text)

profile = get_or_create_profile(data)
profile

{'id': '3916dd67-4795-4f68-9f02-36d914d2094b',
 'created_date': '2022-05-22 17:57:42',
 'last_name': 'Heard',
 'name': 'Amber Heard',
 'birth': '1986-04-22',
 'thumbnail_id': None,
 'thumbnail': None,
 'middle_name': '',
 'sex': None,
 'updated_date': None,
 'faces': [{'id': '729bbed5-e050-4533-aa5d-f8eed2bad53c',
   'created_date': '2022-05-22 18:00:59',
   'photo_id': 'eb9ac457-6f5e-4305-8746-4f9c571bc4b6',
   'location': [79.48835754394531,
    47.92854690551758,
    179.94769287109375,
    182.1373748779297],
   'photo': {'id': 'eb9ac457-6f5e-4305-8746-4f9c571bc4b6',
    'created_date': '2022-05-22 18:00:59',
    'url': 'https://m.media-amazon.com/images/M/MV5BMjA4NDkyODA3M15BMl5BanBnXkFtZTgwMzUzMjYzNzM@.jpg',
    'height': 272,
    'updated_date': None,
    'sha256_hash': '7d37e06be3a276d950aa96253d16c46cd1c18cda2ec45dd0261f4d93b719a159',
    'width': 272},
   'profile': {'id': '3916dd67-4795-4f68-9f02-36d914d2094b',
    'created_date': '2022-05-22 17:57:42',
    'last_name': 'Hea

In [85]:
r = requests.post(
    API_URL + "/photos", json={"url": data["image"]}
)
photo = json.loads(r.text)
if "error" in photo:
    photo = photo["data"]
photo

{'id': 'eb9ac457-6f5e-4305-8746-4f9c571bc4b6',
 'created_date': '2022-05-22 18:00:59',
 'url': 'https://m.media-amazon.com/images/M/MV5BMjA4NDkyODA3M15BMl5BanBnXkFtZTgwMzUzMjYzNzM@.jpg',
 'height': 272,
 'updated_date': None,
 'sha256_hash': '7d37e06be3a276d950aa96253d16c46cd1c18cda2ec45dd0261f4d93b719a159',
 'faces': [{'id': '729bbed5-e050-4533-aa5d-f8eed2bad53c',
   'created_date': '2022-05-22 18:00:59',
   'photo_id': 'eb9ac457-6f5e-4305-8746-4f9c571bc4b6',
   'location': [79.48835754394531,
    47.92854690551758,
    179.94769287109375,
    182.1373748779297],
   'profile': {'id': '3916dd67-4795-4f68-9f02-36d914d2094b',
    'created_date': '2022-05-22 17:57:42',
    'last_name': 'Heard',
    'name': 'Amber Heard',
    'birth': '1986-04-22',
    'attributes': [{'id': '6810d592-7111-43d5-91ad-3ab7900787b3',
      'created_date': '2022-05-22 17:57:42',
      'key': 'birth_place',
      'updated_date': None,
      'profile_id': '3916dd67-4795-4f68-9f02-36d914d2094b',
      'value': 'Au

In [76]:
for face in photo["faces"]:
    requests.patch(API_URL + f'/faces/{face["id"]}', json={"profile_id": profile["id"]})


In [89]:
profile["updated_date"] != profile["created_date"]

True